<a href="https://colab.research.google.com/github/SergeyDani/boilerplate-rock-paper-scissors/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
df=dataset
#convert catagorical data to numeric data.
df_catagorical = df.select_dtypes(include=['object']).copy()

vocab_dict = {"sex":{"male":0,"female":1},"smoker":{"yes":1,"no":0},"region":{"northeast":3,"northwest":2,"southwest":1,"southeast":0}}
df_2 = df.copy().replace(vocab_dict)
#Standardize data used before training the model.
scaler = MinMaxScaler().fit(df_2)
df_3 = scaler.transform(df_2)
#put scaled data into a dataframe
df_3 = pd.DataFrame(data=df_3,columns=['age','sex','bmi','children','smoker','region','expenses'])
df_3.head()
#Split data,labels into testing and training data,labels.
features_train, features_test, target_train, target_test = train_test_split(
df_3, df_3.pop('expenses'), test_size=0.20, random_state=0)


In [ ]:
#construct model
model = keras.Sequential()

model.add(layers.Dense(units=32,input_shape=(features_train.shape[1],)))
model.add(layers.Activation('relu'))
model.add(layers.Dense(1))

optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001)

model.compile(loss="mse",optimizer=optimizer ,metrics=["mse","mae"])

In [ ]:
#the model is now trained on training data and tested against test data.
history = model.fit(features_train, target_train,epochs=10,verbose=2,batch_size=200,validation_data=(features_test, target_test))


In [ ]:
#set varible names for test
test_dataset = features_test
test_labels = target_test

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)